In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH_EVENTOS    = "data\\eventos.csv"
PATH_PAGOS      = "data\\pagos.csv"
PATH_USUARIOS   = "data\\usuarios.csv"

In [3]:
eventos     = pd.read_csv(PATH_EVENTOS)
pagos       = pd.read_csv(PATH_PAGOS)
usuarios    = pd.read_csv(PATH_USUARIOS)

In [4]:
print(f"{eventos.shape=}")
print(f"{pagos.shape=}")
print(f"{usuarios.shape=}")

eventos.shape=(25293, 4)
pagos.shape=(20000, 6)
usuarios.shape=(5000, 4)


In [5]:
eventos.head()

,user_id,fecha_evento,tipo_evento,dispositivo
0,1,2023-02-06,login,Android
1,1,2023-01-25,compra,Android
2,1,2024-03-27,compra,Web
3,1,2023-02-03,login,Android
4,1,2023-04-29,compra,Web


In [6]:
pagos.head()

,pago_id,user_id,fecha_pago,monto,metodo_pago,fraude
0,1,3387,2024-01-11,25.44,Tarjeta Crédito,0
1,2,3892,2023-07-05,18.53,Tarjeta Crédito,0
2,3,2743,2023-06-28,35.90,Tarjeta Crédito,0
3,4,2935,2023-09-07,0.30,Tarjeta Crédito,0
4,5,3972,2024-06-05,1.48,Débito,0


In [7]:
usuarios.head()

,user_id,pais,fecha_registro,canal_adquisicion
0,1,México,2022-07-24,Orgánico
1,2,Colombia,2023-12-03,Orgánico
2,3,Perú,2023-03-03,Referido
3,4,Perú,2022-03-16,Facebook
4,5,Colombia,2022-06-12,Referido


----------------------------------

# 1. SQL & Modelado de datos

## Ejercicio 1

In [8]:
pagos_por_usuario = pagos.groupby("user_id").agg(ltv_user=("monto", "sum")).reset_index()
usarios_ltv = usuarios.merge(pagos_por_usuario, how="left", on="user_id")
usarios_ltv.groupby("canal_adquisicion").agg(ltv_promedio=("ltv_user", "mean")).reset_index().sort_values("ltv_promedio")

,canal_adquisicion,ltv_promedio
0,Facebook,203.804739
1,Google Ads,204.892044
3,Referido,205.890504
2,Orgánico,208.425704


## Ejercicio 2

In [30]:
eventos["fecha_evento"] = pd.to_datetime(eventos["fecha_evento"], format="%Y-%m-%d")
fecha_lim = eventos["fecha_evento"].max() - pd.Timedelta(days=90)
usarios_activos = eventos.loc[eventos["fecha_evento"] >= fecha_lim, "user_id"].unique()

In [18]:
metodos = pagos.groupby("user_id").agg(metodos_distintos=("metodo_pago", lambda x: len(np.unique(x)))).reset_index()

In [38]:
usuarios_activos_metodos = metodos.loc[(metodos["user_id"].isin(usarios_activos)) & (metodos["metodos_distintos"] >= 2), "user_id"]

In [40]:
usuarios[usuarios["user_id"].isin(usuarios_activos_metodos)].reset_index(drop=True)

,user_id,pais,fecha_registro,canal_adquisicion
0,4,Perú,2022-03-16,Facebook
1,6,Chile,2022-05-10,Referido
2,11,Colombia,2024-02-16,Referido
3,12,Argentina,2022-06-17,Orgánico
4,13,Colombia,2023-02-10,Orgánico
...,...,...,...,...
2101,4992,Perú,2022-05-01,Orgánico
2102,4996,Perú,2024-02-09,Facebook
2103,4997,Perú,2022-11-09,Google Ads
2104,4998,Chile,2023-09-21,Facebook
